# Setup

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook
import gdax
%matplotlib inline  

In [2]:
import plotly.plotly as py
import plotly.graph_objs as go
import plotly

In [3]:
from datetime import datetime
from datetime import timedelta
import calendar

In [4]:
plotly.tools.set_credentials_file(username='gasia4444', api_key='rC8FB3pveGzyME366dL7')

# Read Data

In [5]:
df = pd.read_csv('bitcoin_data_five_minute_2016_2018.csv', parse_dates = ['timestamp']).set_index('timestamp')

In [6]:
df.sort_index(inplace=True)

In [7]:
dd = df
data = [
       go.Scatter(
          x = dd.index,
          y = dd.low,
          mode='lines',
          name='low'),
    
        go.Scatter(
          x = dd.index,
          y = dd.high,
          mode='lines',
          name='high'),
    
        go.Scatter(
          x = dd.index,
          y = dd.open,
          mode='lines',
          name='open'),
    
        go.Scatter(
          x = dd.index,
          y = dd.close,
          mode='lines',
          name='close'),
    
        go.Scatter(
          x = dd.index,
          y = dd.volume,
          mode='lines',
          name='volume'),
    
    ]
py.iplot(data, filename = 'gasia')

/home/gasia/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:224: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~gasia4444/0 or inside your plot.ly account where it is named 'gasia'


/home/gasia/anaconda3/lib/python3.6/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Too Long



In [8]:
dd.tail(5)

,time,low,high,open,close,volume
timestamp,,,,,,
2018-05-19 11:35:00,1526729700,8244.08,8244.40,8244.08,8244.40,2.240500
2018-05-19 11:40:00,1526730000,8244.39,8245.00,8244.40,8245.00,3.743579
2018-05-19 11:45:00,1526730300,8244.99,8269.34,8245.00,8269.34,24.687894
2018-05-19 11:50:00,1526730600,8269.34,8274.00,8269.34,8274.00,9.368873
2018-05-19 11:55:00,1526730900,8270.84,8368.31,8273.99,8368.31,257.014080


-----------------

# Detect Minuimum Maximum

a point is considered a maximum peak if it has the maximal value, and was preceded (to the left) by a value lower by delta, and followed (to the right) by a value lower by delta. 

In [9]:
import sys
from numpy import NaN, Inf, arange, isscalar, asarray, array

def det_min_max(v, delta, x = None):

    maxtab = []
    mintab = []
       
    if x is None:
        x = arange(len(v))
    
    v = asarray(v)
    
    if len(v) != len(x):
        sys.exit('x, v must have same length')
    
    if not isscalar(delta):
        sys.exit('delta is not scalar')
    
    if delta <= 0:
        sys.exit('delta is not positive')
    
    mn, mx = Inf, -Inf
    mnpos, mxpos = NaN, NaN
    
    lookformax = True
    for i in arange(len(v)):
        this = v[i]
        if this > mx:
            mx = this
            mxpos = x[i]
        if this < mn:
            mn = this
            mnpos = x[i]
        
        if lookformax:
            if this < mx-delta:
                maxtab.append((mxpos, mx))
                mn = this
                mnpos = x[i]
                lookformax = False
        else:
            if this > mn+delta:
                mintab.append((mnpos, mn))
                mx = this
                mxpos = x[i]
                lookformax = True

    return array(maxtab), array(mintab)

### apply

In [10]:
df = pd.read_csv('bitcoin_data_five_minute_2016_2018.csv', parse_dates = ['timestamp'])
df.sort_values('timestamp', inplace=True)

In [11]:
series = df['close'].values
maxtab, mintab = det_min_max(series,1000)
df = df.set_index('timestamp')

In [22]:
data = [    
       go.Scatter(
          x = df.iloc[array(maxtab)[:,0]].index.values,
          y = array(maxtab)[:,1],
          mode='markers',
         marker = dict(
             size = 8,
            color = '#229954',
            line = dict(
                width = 1,
                color = 'rgb(0, 0, 0)'),),
        name='maximum'),
    
        go.Scatter(
          x = df.iloc[array(mintab)[:,0]].index.values,
          y = array(mintab)[:,1],
          mode='markers',
          marker = dict(
            size = 8,
            color = '#CB4335',
            line = dict(
                width = 1,
                color = 'rgb(0, 0, 0)')),
        
          name='minimum'),
        
    go.Scatter(
          x = df.index.values,
          y = df.close.values,
          mode='lines',
          line = dict(color = '#2E86C1'),
          name='btc ts'),
          

    ]
py.iplot(data, filename = 'gasia')

/home/gasia/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:224: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~gasia4444/0 or inside your plot.ly account where it is named 'gasia'


/home/gasia/anaconda3/lib/python3.6/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Too Long



In [28]:
pd.DataFrame(array(maxtab)[:,1] , index=df.iloc[array(maxtab)[:,0]].index, columns=['data']).to_csv('btc_max.csv')
pd.DataFrame(array(mintab)[:,1] , index=df.iloc[array(mintab)[:,0]].index, columns=['data']).to_csv('btc_min.csv')
